## Topic Modeling Using LDA (Latent Dirichlet Allocation)
Here 20 Newsgroup data set is processed to find the relevant topics by applying LDA clustering algorithm.

* Spacy is used for text processing.
* Scikit is used for topic modeling (LDA).
* pyLDAvis is used for visualizing topics.

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

from spacy import load

import pandas as pd
import re

In [2]:
# Spacy is loaded for text processing tasks.
nlp = load('en')

In [3]:
# Load Corpus
corpus= pd.read_csv('20newsgroup.csv', encoding='utf-8')
data_set = corpus['Text']
print("Size of data set is %s." %str(len(data_set)))

Size of data set is 19955.


In [4]:
def _nlp_layer(x_series):
    print("Performing text normalization.")
    contractions_list = {
        "ain't": "am not",
        "aren't": "are not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he would",
        "he'd've": "he would have",
        "he'll": "he will",
        "he'll've": "he will have",
        "he's": "he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how is",
        "i'd": "i would",
        "i'd've": "i would have",
        "i'll": "i will",
        "i'll've": "i will have",
        "i'm": "i am",
        "i'am": "i am",
        "i've": "i have",
        "isn't": "is not",
        "it'd": "it had",
        "it'd've": "it would have",
        "it'll": "it will",
        "it'll've": "it will have",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she would",
        "she'd've": "she would have",
        "she'll": "she will",
        "she'll've": "she will have",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so is",
        "that'd": "that would",
        "that'd've": "that would have",
        "that's": "that is",
        "there'd": "there had",
        "there'd've": "there would have",
        "there's": "there is",
        "they'd": "they would",
        "they'd've": "they would have",
        "they'll": "they will",
        "they'll've": "they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we had",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what will",
        "what'll've": "what will have",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "when's": "when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where is",
        "where've": "where have",
        "who'll": "who will",
        "who'll've": "who will have",
        "who's": "who is",
        "who've": "who have",
        "why's": "why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'alls": "you alls",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you had",
        "you'd've": "you would have",
        "you'll": "you you will",
        "you'll've": "you you will have",
        "you're": "you are",
        "you've": "you have"
    }

    c_re = re.compile('(%s)' % '|'.join(contractions_list.keys()))

    def expand_contractions(text, c_re_=c_re):
        def replace(match):
            return contractions_list[match.group(0)]

        return c_re_.sub(replace, text)

    x = [re.sub('[^0-9a-z\' ]+', ' ', item.lower()).split() for item in x_series]
    processed_x = []
    for eachQuery in x:
        query = []
        for eachToken in eachQuery:
            expanded_token = expand_contractions(eachToken)
            # Applied to normalize words like word's, mother's, 1980's etc.
            if '\'' in expanded_token:
                expanded_token = expanded_token.strip('\'s')
            # Replace alpha-numeric with special token 'Special_Tok'
            if expanded_token.isalnum() and not expanded_token.isalpha() and not expanded_token.isdigit():
                expanded_token = 'SpecialTok'
            # Replace all numerals with special token - 'NUM'
            if expanded_token.isdigit():
                expanded_token = 'NUM'
            query.extend(expanded_token.split())
        processed_x.append(query)
    return processed_x

In [5]:
data_set_processed = _nlp_layer(data_set)
corpus_for_tf = []
for each_document in data_set_processed:
    each_file = ""
    each_file += " ".join(each_token for each_token in each_document)
    corpus_for_tf.append(each_file)

Performing text normalization.


### Document-term matrix is generated using CountVectorizer provided by Scikit Learn. The maximum features is limited to most frequent 1 lakh tokens, which will be the vocabulary size. The corpus originally consists of 401,467 tokens.

In [6]:
tf_vectorizer = CountVectorizer(ngram_range=(1,1),
                                max_features=100000,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(corpus_for_tf)

In [7]:
print("Fitting LDA model with term frequency features, n_samples=%d and n_features=%d." % (tf.shape[0], tf.shape[1]))
lda = LatentDirichletAllocation(n_components=20, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)

Fitting LDA model with term frequency features, n_samples=19955 and n_features=100000.


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=20, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [8]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ",".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message,"\n")

In [9]:
print("Topics in LDA model and associated terms:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 20)

Topics in LDA model and associated terms:
Topic #0: specialtok,ax,num,max,pl,db,cx,bhj,giz,wm,ah,sl,bh,mq,gk,chz,bxn,mv,hz,lk 

Topic #1: armenian,turkish,armenians,men,said,children,people,women,arab,muslims,muslim,armenia,sex,sexual,history,cancer,islam,today,party,years 

Topic #2: virginia,henry,myers,harvard,spencer,acc,hess,ryan,liver,erik,vote,murdoch,motto,waldrop,zoology,dzkriz,uio,galaxy,ndet,bob 

Topic #3: edu,atf,mil,umd,navy,ra,cs,ai,uga,michael,colostate,boeing,wam,maine,acns,racism,rpi,john,div,oakland 

Topic #4: specialtok,com,edu,writes,article,uiuc,netcom,cso,sin,andrew,news,cmu,hell,brian,mary,david,isc,opinions,ca,steve 

Topic #5: nra,tires,motorcycles,circuits,arf,tire,lehigh,seats,lunar,gretzky,nsmca,obo,kilometers,logo,tampa,moncton,transportation,adirondack,champs,kirlian 

Topic #6: num,specialtok,new,israel,christ,edu,university,april,guns,national,american,period,bike,st,war,team,ed,john,total,league 

Topic #7: people,god,government,say,think,writes,belie

### Below the topics are visualized on a 2D plane on the left side and the term distribution is shown on the right side.

In [10]:
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/home/gopal/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12     0.312840 -0.198653       1        1  27.959027
17     0.319900 -0.000490       2        1  17.633826
6      0.272814  0.130261       3        1  17.265638
7      0.259393 -0.265139       4        1  14.400728
0      0.172718  0.351510       5        1   7.875437
1      0.034083 -0.144237       6        1   2.504301
4      0.095046  0.054943       7        1   2.405736
8      0.120098  0.176692       8        1   2.261743
9     -0.072621 -0.016257       9        1   0.920819
18    -0.114750 -0.008009      10        1   0.850465
15    -0.137739  0.000268      11        1   0.747464
3     -0.101195 -0.011710      12        1   0.679996
19    -0.135770 -0.006028      13        1   0.633379
2     -0.136463 -0.009500      14        1   0.602730
16    -0.146346 -0.002421      15        1   0.577604
14    -0.144071 -0.007333      16        1   0.577070
13    -0.146375 -0.016941      17        1   0.568377
10    -0.149795 -0.008229      18        1   0.525048
11    -0.148587 -0.011266      19        1   0.523108
5     -0.153179 -0.007461      20        1   0.487504, topic_info=      Category           Freq            Term          Total  loglift  logprob
term                                                                          
59845  Default  244773.000000             num  244773.000000  30.0000  30.0000
80851  Default  124137.000000      specialtok  124137.000000  29.0000  29.0000
6226   Default   60455.000000              ax   60455.000000  28.0000  28.0000
25440  Default   21362.000000             edu   21362.000000  27.0000  27.0000
15729  Default   13107.000000             com   13107.000000  26.0000  26.0000
96394  Default   14274.000000          writes   14274.000000  25.0000  25.0000
4880   Default   12252.000000         article   12252.000000  24.0000  24.0000
64165  Default   10173.000000          people   10173.000000  23.0000  23.0000
52124  Default    4727.000000             max    4727.000000  22.0000  22.0000
34428  Default    5143.000000             god    5143.000000  21.0000  21.0000
95527  Default    2895.000000         windows    2895.000000  20.0000  20.0000
24240  Default    2371.000000           drive    2371.000000  19.0000  19.0000
75353  Default    4270.000000            said    4270.000000  18.0000  18.0000
18738  Default    2866.000000              cs    2866.000000  17.0000  17.0000
23560  Default    6948.000000            does    6948.000000  16.0000  16.0000
47312  Default    8874.000000            know    8874.000000  15.0000  15.0000
45583  Default    9734.000000            just    9734.000000  14.0000  14.0000
86454  Default    7818.000000           think    7818.000000  13.0000  13.0000
91599  Default    6789.000000             use    6789.000000  12.0000  12.0000
23828  Default    1841.000000             dos    1841.000000  11.0000  11.0000
67694  Default    3463.000000         problem    3463.000000  10.0000  10.0000
23681  Default   10261.000000             don   10261.000000   9.0000   9.0000
49390  Default    2012.000000            line    2012.000000   8.0000   8.0000
12269  Default    1601.000000            card    1601.000000   7.0000   7.0000
34776  Default    3319.000000      government    3319.000000   6.0000   6.0000
36686  Default    1862.000000            hard    1862.000000   5.0000   5.0000
75927  Default    4768.000000             say    4768.000000   4.0000   4.0000
22084  Default    4566.000000             did    4566.000000   3.0000   3.0000
30230  Default    3007.000000            file    3007.000000   2.0000   2.0000
4054   Default    1105.000000           apple    1105.000000   1.0000   1.0000
...        ...            ...             ...            ...      ...      ...
76880  Topic20      85.457585           seats      86.291981   5.3139  -5.1609
35250  Topic20      80.475730         gretzky      81.310125   5.3133  -5.2209
5972